In [1]:
import pandas as pd
import numpy as np

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

In [2]:
df = pd.read_csv("bank_marketing.csv")

In [3]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


### Create the economics file

`economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [4]:
df.client_id.duplicated().any()

np.False_

In [5]:
economics_df = df[['client_id','cons_price_idx','euribor_three_months']]
economics_df.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [6]:
economics_df.dtypes

client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object

In [7]:
economics_df.to_csv('economics.csv', index=False)

### Create the client file

`client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

In [8]:
client_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

In [9]:
client_df.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [10]:
client_df.loc[:,'job'] = client_df.loc[:, 'job'].str.replace('.','_')
client_df.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin_,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [11]:
client_df.loc[:,'education'] = client_df.loc[:,'education'].str.replace('.','_')

client_df.loc[client_df['education'] == 'unknown','education'] = np.nan

client_df.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,no,no
1,1,57,services,married,high_school,unknown,no
2,2,37,services,married,high_school,no,yes
3,3,40,admin_,married,basic_6y,no,no
4,4,56,services,married,high_school,no,no


In [12]:
client_df[client_df['credit_default'] == 'yes']

,client_id,age,job,marital,education,credit_default,mortgage
21580,21580,48,technician,married,professional_course,yes,no
21581,21581,48,technician,married,professional_course,yes,yes
24866,24866,31,unemployed,married,high_school,yes,no


In [13]:
client_df.loc[client_df['credit_default'] == 'yes','credit_default'] = 1
client_df.loc[client_df['credit_default'] != 1,'credit_default'] = 0

client_df[client_df['credit_default'] == 1]

,client_id,age,job,marital,education,credit_default,mortgage
21580,21580,48,technician,married,professional_course,1,no
21581,21581,48,technician,married,professional_course,1,yes
24866,24866,31,unemployed,married,high_school,1,no


In [14]:
client_df['credit_default'] = client_df['credit_default'].astype(bool)

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\2466300065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df['credit_default'] = client_df['credit_default'].astype(bool)


In [15]:
client_df.loc[client_df['mortgage'] == 'yes','mortgage'] = True
client_df.loc[client_df['mortgage'] != True,'mortgage'] = False

client_df[client_df['mortgage'] == False]

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
5,5,45,services,married,basic_9y,False,False
...,...,...,...,...,...,...,...
41168,41168,38,entrepreneur,married,university_degree,False,False
41178,41178,62,retired,married,university_degree,False,False
41180,41180,36,admin_,married,university_degree,False,False
41184,41184,46,blue-collar,married,professional_course,False,False


In [16]:
client_df['mortgage'] = client_df['mortgage'].astype(bool)

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\3166492399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df['mortgage'] = client_df['mortgage'].astype(bool)


In [17]:
client_df['mortgage'].value_counts()

mortgage
True     21576
False    19612
Name: count, dtype: int64

In [18]:
client_df.dtypes

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object

In [19]:
client_df.to_csv('client.csv', index=False)

### Create campaign file

`campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

In [20]:
campaign_df = df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','month', 'day']]

In [21]:
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,0,nonexistent,no,may,13
1,1,1,149,0,nonexistent,no,may,19
2,2,1,226,0,nonexistent,no,may,23
3,3,1,151,0,nonexistent,no,may,27
4,4,1,307,0,nonexistent,no,may,3


In [22]:
campaign_df.loc[campaign_df['previous_outcome'] == 'success','previous_outcome'] = 1
campaign_df.loc[campaign_df['previous_outcome'] != 1,'previous_outcome'] = 0
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].astype(bool)

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\1419740702.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['previous_outcome'] = campaign_df['previous_outcome'].astype(bool)


In [23]:
campaign_df.loc[campaign_df['campaign_outcome'] == 'yes','campaign_outcome'] = 1
campaign_df.loc[campaign_df['campaign_outcome'] != 1,'campaign_outcome'] = 0
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].astype(bool)

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\2249884093.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].astype(bool)


In [34]:
campaign_df['month'].value_counts()

month
05    13769
07     7174
08     6178
06     5318
11     4101
04     2632
10      718
09      570
03      546
12      182
Name: count, dtype: int64

In [32]:
months = {'jan':'01',
          'feb':'02',
          'mar':'03',
          'apr':'04',
          'may':'05',
          'jun':'06',
          'jul':'07',
          'aug':'08',
          'sep':'09',
          'oct':'10',
          'nov':'11',
          'dec':'12'}

In [33]:
campaign_df.month = campaign_df.month.map(months)

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\346959714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df.month = campaign_df.month.map(months)


In [25]:
campaign_df['year'] = '2022'

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\2009743090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['year'] = '2022'


In [ ]:
campaign_df['day'] = campaign_df['day'].astype(str)

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\3390391990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['day'] = campaign_df['day'].astype(str)


In [ ]:
campaign_df['last_contact_date'] = campaign_df['year'] + '-' + campaign_df['month'] + '-' + campaign_df['day']
campaign_df['last_contact_date'] = pd.to_datetime(campaign_df['last_contact_date'])

C:\Users\Arttu\AppData\Local\Temp\ipykernel_22456\1379155535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['last_contact_date'] = campaign_df['year'] + '-' + campaign_df['month'] + '-' + campaign_df['day']


,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,year,last_contact_date
0,0,1,261,0,False,False,05,13,2022,2022-05-13
1,1,1,149,0,False,False,05,19,2022,2022-05-19
2,2,1,226,0,False,False,05,23,2022,2022-05-23
3,3,1,151,0,False,False,05,27,2022,2022-05-27
4,4,1,307,0,False,False,05,3,2022,2022-05-3


In [40]:
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,year,last_contact_date
0,0,1,261,0,False,False,05,13,2022,2022-05-13
1,1,1,149,0,False,False,05,19,2022,2022-05-19
2,2,1,226,0,False,False,05,23,2022,2022-05-23
3,3,1,151,0,False,False,05,27,2022,2022-05-27
4,4,1,307,0,False,False,05,3,2022,2022-05-03


In [41]:
campaign_df = campaign_df.drop(['year','month','day'], axis = 1)

In [42]:
campaign_df.dtypes

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                        bool
campaign_outcome                        bool
last_contact_date             datetime64[ns]
dtype: object

In [43]:
campaign_df.to_csv('campaign.csv', index=False)